# IMC Thresholding.

This is an interactive tool to manually set the tresholds for each IMC channels.

You should set up the data to load, and if necessary, some custom functions

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pickle
import ipywidgets as widgets
from IPython.display import display
import os
from helper_functions import *
import app

### Load data
Load your data here. Change ['im'] to what suits your data.

In [2]:
picklename = 'eg_data.p'
data = pickle.load(open(picklename,'rb'))

fn_ls = list(data.keys())
markers = list(data[fn_ls[0]]['im'].keys())

## log normalization if not done yet
# for fn in data.keys():
#     for m in data[list(data.keys())[0]]['im'].keys():
#         data[fn]['im'][m] = np.log(data[fn]['im'][m]+1)

#### Output directory

In [3]:
# output directory
outdir = 'preprocessing'
os.makedirs(outdir, exist_ok=True)

#### Load previous thresholds
If you have previous threshold, load them as pandas dataframe here, where index = ROIs and columns = markers.

 Otherwise, leave them as None.

In [4]:
cmins_table = None
cmaxs_table = None
try:
    cmins_table = pd.read_csv(os.path.join(outdir,'cmins.csv'), index_col=0)
    cmaxs_table = pd.read_csv(os.path.join(outdir,'cmaxs.csv'), index_col=0) 
    print(cmins_table)
except:
    pass

       CD3  CD68  Melanoma  DNA1  CD45
roi4  3.72   NaN       NaN   NaN   NaN
roi6   NaN   NaN       NaN   NaN   NaN


In the example, the CD3 channel of roi4 has been thresholded. Therefore the app will load the thresholds for this channel. 

### Custom functions
Modify these functions based on your data type or data structure.

Add preprocessing steps in readchannel(), and postprocessing steps in savetiff().

In [5]:
def readchannel(fn, m, data=data, p=99):
    ''' Read the channel {m} for image {fn} and cap it at {p} percentile. '''
    im = data[fn]['im'][m]
    im[im > np.percentile(nonzero(im), p)] =  np.percentile(nonzero(im), p)
    return im

def savetiff(fn, m, im, cmin, cmax, outdir):
    '''Save current channel as {outdir}/{fn}/{m}.tiff. '''

    im = im.copy()

    ## threshold with cmin
    im[im < cmin] = 0
    
    ## cap with cmax
    im[im > cmax] = cmax
    
    ## to normalize to (0,1) by cmax:
    # im = im/max(cmax, 1e-6)

    ## output
    os.makedirs(os.path.join(outdir, fn), exist_ok=True)
    Image.fromarray(im).save(os.path.join(outdir, fn, f"{m}.tiff"))
    

## App
(You should fold this.)

In [6]:
import app
gui = app.ThresholdGUI(
    data = data, 
    fn_ls = fn_ls, 
    markers = markers, 
    outdir = outdir, 
    func_dict = {'readchannel':readchannel, 'savetiff':savetiff}, 
    cmins_table = cmins_table, 
    cmaxs_table = cmaxs_table
    )